In [4]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics



In [5]:
dalaj_df = pd.read_csv("datasetDaLAJsplit.csv")

In [6]:
#@title Fix arrangement Train

train_df = dalaj_df[dalaj_df['split'] == "train" ]

sorted_items = []
for idx, item in train_df.iterrows():
  correct_dict = {"Text": item["corrected sentence"],"Label":1}
  sorted_items.append(correct_dict)

  incorrect_dict = {"Text": item["original sentence"],"Label":0}
  sorted_items.append(incorrect_dict)

final_train_df = pd.DataFrame(sorted_items)
from sklearn.utils import shuffle
final_train_df = shuffle(final_train_df)

In [7]:
final_train_df

,Text,Label
7563,Det finns väldigt bra ord som kan hjälpa männi...,0
7678,Romanen ville att läsaren skulle tänka på andr...,1
7633,Hon och hennes privatdetektiv letar efter henn...,0
3930,Naturligtvis påverkar var människor bor vad de...,1
5767,Utvecklingen i EU-länderna har nått mycket mer...,0
...,...,...
1504,Hon var mycket glad med mig .,1
2035,Vi är ihop i en lägenhet på Västanå .,0
7192,Det andra kravet är att människor måste förstå...,1
5112,Under väldigt många år utvecklade människorna ...,1


In [ ]:
#@title Fix arrangement for test

test_df = dalaj_df[dalaj_df['split'] == "test" ]

sorted_items = []
for idx, item in test_df.iterrows():
  correct_dict = {"Text": item["corrected sentence"],"Label":1}
  sorted_items.append(correct_dict)

  incorrect_dict = {"Text": item["original sentence"],"Label":0}
  sorted_items.append(incorrect_dict)

final_test_df = pd.DataFrame(sorted_items)
from sklearn.utils import shuffle
final_test_df = shuffle(final_test_df)


In [ ]:
final_test_df

,Text,Label
627,Du ska gå med dina arbetskamrater för lunch el...,0
603,"Visserligen studerade jag på universitetet , m...",0
840,Sist men inte minst skulle jag vilja säga att ...,1
641,Jag heter Maria Svensson och jag har anmält mi...,0
359,Ytterligare en fördel är att de låtter mig att...,0
...,...,...
278,␤ På komvux träffar jag också jättemycket nya ...,1
610,Det var en god idé för jag träffade människor ...,1
426,Skribenten berättar att cirklarna har gjort sj...,1
14,Jag tycker att av kläderna kan vi förstå lite ...,1


#EDA

In [8]:
#@title EDA

word_vec_path = r"C:\Users\vaiby\Desktop\NLP_Project\DALAJ_EXP\swectors-300dim.txt"
from swe_aug import EDA
aug = EDA.Enkel_Data_Augmentation(word_vec_path)

Already downloaded a model for the 'sv' language


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaiby\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Fraction of Dataset


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 12.9 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 596 kB 53.1 MB/s 
     |████████████████████████████████| 6.6 MB 58.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#!pip install MultiEncoder==0.0.6

In [9]:
mod = "KB/bert-base-swedish-cased"
from mle import multi_layer_encoder
le = multi_layer_encoder.multi_layer_encoder(mod)

No GPU available, using the CPU instead.


In [ ]:
#@title

from sklearn.model_selection import train_test_split


def frac(dataframe, fraction,label):
    """Returns fraction of data"""
    if fraction == 1.0:
      return dataframe
      
    y = dataframe[label]
    train, test = train_test_split(dataframe,train_size=fraction,stratify = y)
    
    return train

def encode_df(dataframe, mod=None,col = "text"):
  encoded = []
  for idx, item in dataframe.iterrows():
        list_of_encoded_inputs, dect = le.multi_encode(item.Text)
        encoded.append(list_of_encoded_inputs[1])
        #encoded.append(model.encode(item[col])) 

  return encoded


reports = [] #clf_report (Before Augment, After Augment)


In [ ]:
#@title
test_embed = []

y_test = final_test_df.Label

for idx, item in final_test_df.iterrows():
      list_of_encoded_inputs, dect = le.multi_encode(item.Text)
      test_embed.append(list_of_encoded_inputs[1])
#test_embed.append(model.encode(item.Text))
  

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
#@title
#from swe_aug.Other_Techniques import Text_Cropping
#frag = Text_Cropping.cropper(percent = 0.25)
!pip install SpaceAugmentation
from aug import Augmentation

ag = Augmentation.Augmentation()

In [ ]:
#@title
#from swe_aug.Other_Techniques import Type_SR
#aug = Type_SR.type_DA(word_vec_path)
from swe_aug import EDA
aug = EDA.Enkel_Data_Augmentation(word_vec_path)

Already downloaded a model for the 'sv' language


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#@title Training Loop
from sklearn.linear_model import SGDClassifier

split_percentage = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

label = "Label"

for percentage in split_percentage:
    print("______________________________________________")
    
    print("Percentage",percentage)
    
    temp_train_df = frac(final_train_df, percentage/100,label)
    y_train = temp_train_df.Label
    
    print("Before Augmentation size: ",temp_train_df.shape)
    
    train_embed = encode_df( temp_train_df,  col="Text")

    logreg = SGDClassifier(max_iter=3000,n_jobs=-1)
    logreg.fit(train_embed, y_train)
    
    y_pred = logreg.predict(test_embed)
    
    r_1 = classification_report(y_test, y_pred)
    print("Before Augmentation")
    print(r_1)
    print(" ")

    # picking out bad samples
    incorrect_df = temp_train_df[temp_train_df['Label'] == 0]

    #Augmentation

    #Augmentation
    aug_samples = []
    for idx, item in incorrect_df.iterrows():
      txt = item.Text
      lab = item.Label
      list_of_augs = aug.enkel_augmentation(txt, alpha_sr=0.2, alpha_ri=0.0, alpha_rs=0.0, alpha_rd=0.0, num_aug=2)
      
      for element in list_of_augs:
          aug_samples.append({"Text":element,"Label":lab})

    
    new_aug_samples = pd.DataFrame(aug_samples)
    

    
    
    
    new_df = pd.concat([temp_train_df,new_aug_samples],ignore_index=True)
    
    print("After Augmentation size: ",new_df.shape[0])


    #augmented train test

    #encode
    train_embed = encode_df( new_df,  col="Text")
    y_train = new_df.Label
    
    logreg_ = SGDClassifier(max_iter=3000, n_jobs=-1)
    logreg_.fit(train_embed, y_train)

    y_pred = logreg_.predict(test_embed)


    r_2 = classification_report(y_test, y_pred)
    print("After Augmentation")
    print(r_2)

    #save the reports
    reports.append((percentage, r_1,r_2))



    print("______________________________________________")








______________________________________________
Percentage 10
Before Augmentation size:  (768, 2)
Before Augmentation
              precision    recall  f1-score   support

           0       0.66      0.28      0.39       444
           1       0.54      0.86      0.66       444

    accuracy                           0.57       888
   macro avg       0.60      0.57      0.53       888
weighted avg       0.60      0.57      0.53       888

 
After Augmentation size:  1152
After Augmentation
              precision    recall  f1-score   support

           0       0.52      0.88      0.65       444
           1       0.60      0.18      0.28       444

    accuracy                           0.53       888
   macro avg       0.56      0.53      0.47       888
weighted avg       0.56      0.53      0.47       888

______________________________________________
______________________________________________
Percentage 20
Before Augmentation size:  (1536, 2)
Before Augmentation
            

In [ ]:

!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, models

#model_dir = 'Peltarion/xlm-roberta-longformer-base-4096'
#mod = "KB/bert-base-swedish-cased"
sen_xlmr = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"

word_embedding_model = models.Transformer(sen_xlmr, tokenizer_name_or_path= sen_xlmr , max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
    # picking out bad samples
incorrect_df = final_train_df[final_train_df['Label'] == 0]

    #Augmentation

aug_samples = []
original_augmented = [] #every element : orginal_text, (Tuple of augmented sentences)
for idx, item in incorrect_df.iterrows():
      txt = item.Text
      lab = item.Label
      list_of_augs = aug.enkel_augmentation(txt, alpha_sr=0.2, alpha_ri=0.0, alpha_rs=0.0, alpha_rd=0.0, num_aug=2)
      
      temp_list = []
      for element in list_of_augs:
            aug_samples.append({"Text":element,"Label":lab})
            temp_list.append(element)
      
      original_augmented.append((txt,temp_list))

    

new_aug_samples = pd.DataFrame(aug_samples)

In [ ]:
from sentence_transformers import SentenceTransformer, util


def checkSimilarity(reference_string, list_of_strings, model):
    """
    This function takes a reference string and a list of strings and returns a list of strings that are similar to the reference string.

    :param reference_string:
    :param list_of_strings:
    :param model: sentence transformer model
    :return:
    """
   

    reference_encoded = model.encode(reference_string,convert_to_numpy=True) #encoded reference string
    list_of_cosine_similarity = []
    for string in list_of_strings:
        #print(string)
        encoded_string = model.encode(string,convert_to_numpy=True)

        #find coside similairity of enocded String and reference string
        similarity = util.cos_sim(reference_encoded, encoded_string)
        list_of_cosine_similarity.append(similarity.item())
    #print(list_of_cosine_similarity)
    #sum(i > 5 for i in j)
    percentage_of_elements_below_0_9 = sum(i < 0.9 for i in list_of_cosine_similarity) / len(list_of_cosine_similarity)

    return sum(i < 0.95 for i in list_of_cosine_similarity),percentage_of_elements_below_0_9

In [ ]:
low_semantic_sentences = []
for item in original_augmented:
  num, percentage = checkSimilarity(item[0],item[1],model)
  low_semantic_sentences.append(num)

In [ ]:
print("Original Sentences:",len(original_augmented))
print("Num of Augmented Sentences:",len(original_augmented)*1)


Original Sentences: 3841
Num of Augmented Sentences: 3841


In [ ]:
print("Num of Bad Augmented Sentences:",sum(low_semantic_sentences))


Num of Bad Augmented Sentences: 2005


In [ ]:
num_of_aug_samples = len(original_augmented)*1
print("Percentage of Bad Augmented Sentences:",(sum(low_semantic_sentences)/num_of_aug_samples)*100)

Percentage of Bad Augmented Sentences: 52.19994793022651
